In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load Dataset

In [ ]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.2),
    transforms.RandomRotation(degrees=20),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=256, shuffle=True, drop_last = True)

testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=256, shuffle=False, drop_last = True)

# Model

In [ ]:
class VGG(nn.Module):
    def __init__(self):
        super(VGG, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),   
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),                                    

            nn.Conv2d(64, 128, kernel_size=3, padding=1),         
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),       
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
        )

        self.classifier = nn.Sequential(
            nn.Linear(4 * 4 * 256, 4096),
            nn.Dropout(0.2),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 4096),
            nn.Dropout(0.2),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 512),
            nn.Dropout(0.2),
            nn.ReLU(inplace=True),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(-1, 4 * 4 * 256)  # Adjusted for the added depth
        x = self.classifier(x)
        return x

# Train Function

In [ ]:
def train(criterion, optimizer, model, num_epochs, device):
    loss_history = []
    test_acc_history = []

    for epoch in range(num_epochs):
        loss_per_epoch = 0
        for images, label in trainloader:
            optimizer.zero_grad()

            images, label = images.to(device), label.to(device)

            outputs = model(images)

            loss = criterion(outputs, label)
            loss.backward()
            optimizer.step()

            loss_per_epoch += loss

        correct = 0
        total = 0

        with torch.no_grad():
            for images, label in testloader:
                images, label = images.to(DEVICE), label.to(DEVICE)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += label.size(0)
                correct += (predicted == label).sum().item()

        print(f"epoch: {epoch}, loss_per_epoch: {loss_per_epoch}, test_accuracy: {correct/total}")
        loss_history.append(loss_per_epoch.cpu().detach().numpy())
        test_acc_history.append((correct * 100/total))

    return loss_history, test_acc_history

In [ ]:
model = VGG()
model = model.to(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)

num_epochs = 20
loss_history, test_acc_history = train(criterion, optimizer, model, num_epochs = num_epochs, device = DEVICE)